In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint
from scipy.signal import decimate

In [ ]:
from scipy.io.wavfile import read


In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)
layers = []

In [ ]:
hz, a = read("/mnt/c/data/side/craigie.wav")
a = np.array(a,dtype=float)
df = pd.DataFrame(a)
df.index = pd.to_timedelta(df.index / hz, unit='s')

df = df.rename(columns={0:'L', 1:'R'})
dfrs = df.resample('1ms').mean()

start = '170s'
end = '260s'
clip = dfrs.loc[start:end, :]

In [ ]:
baseline_spiral = np.linspace(0, 1, n_samples) ** 0.5

In [ ]:
angle_max = np.pi*40
signal_weight = 0.008
n_samples = len(signal)
baseline_spiral = np.linspace(0, 1, n_samples) ** 0.5
angles = np.linspace(0, angle_max ** 2, n_samples) ** 0.5

radii = baseline_spiral + signal * signal_weight
xs = np.cos(thetas) * radii
ys = np.sin(thetas) * radii

In [ ]:


signal = clip['L'].values
signal /= signal.std()
spiral = np.linspace(0, 1, len(signal)) ** 0.5
thetas = np.linspace(0, (np.pi*40) ** 2 , len(signal)) ** 0.5
scale = 0.008
rs = spiral + signal * scale
xs = np.cos(thetas) * rs
ys = np.sin(thetas) * rs
line = LineString(np.flipud(np.stack([xs, ys]).T))
layer = gp.make_like(line, drawbox)
layers.append(layer)

signal = clip['R'].values
signal /= signal.std()
spiral = np.linspace(0, 1, len(signal)) ** 0.5
thetas = np.linspace(0, (np.pi*40) ** 2 , len(signal)) ** 0.5
scale = 0.008
rs = spiral + signal * scale
xs = np.cos(thetas) * rs
ys = np.sin(thetas) * rs
line = LineString(np.flipud(np.stack([xs, ys]).T))
layer = gp.make_like(line, drawbox)
layers.append(layer)

In [ ]:
hz, a = read("/mnt/c/data/side/craigie.wav")
a = np.array(a,dtype=float)
df = pd.DataFrame(a)
df.index = pd.to_timedelta(df.index / hz, unit='s')

df = df.rename(columns={0:'L', 1:'R'})
dfrs = df

start = '170s'
end = '260s'
clip = dfrs.loc[start:end, :]
short_clip = dfrs.loc['184s':'195s', :]

# signal = clip['L'].values
# signal = decimate(signal, 5)
# signal = decimate(signal, 5)
# signal /= signal.std()
# spiral = np.linspace(0, 1, len(signal)) ** 0.5
# thetas = np.linspace(0, (np.pi*40) ** 2 , len(signal)) ** 0.5
# scale = 0.008
# rs = spiral + signal * scale
# xs = np.cos(thetas) * rs
# ys = np.sin(thetas) * rs
# line = LineString(np.flipud(np.stack([xs, ys]).T))
# layer = gp.make_like(line, drawbox)
# layers.append(layer)

signal = clip['R'].values
signal = decimate(signal, 7)
signal = decimate(signal, 7)
signal /= signal.std()
spiral = np.linspace(0, 1, len(signal)) ** 0.5
thetas = np.linspace(0, (np.pi*40) ** 2 , len(signal)) ** 0.5
scale = 0.008
rs = spiral + signal * scale
xs = np.cos(thetas) * rs
ys = np.sin(thetas) * rs
line = LineString(np.flipud(np.stack([xs, ys]).T))
layer = gp.make_like(line, drawbox)
layers.append(layer)

In [ ]:
theta_max = np.pi * 40

In [ ]:
n_samples = len(signal)
spiral = np.linspace(0, 1, n_samples) ** 0.5
thetas = np.linspace(0, theta_max ** 2 , n_samples) ** 0.5
signal_weight = 0.008
rs = spiral + signal * signal_weight
xs = np.cos(thetas) * rs
ys = np.sin(thetas) * rs

In [ ]:
[len(layer.coords) for layer in layers]

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')
sk.vpype('linesimplify')
# sk.vpype('linesimplify')
# sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/254_use_sound.svg')